# Exercício 1 - Trabalho Prático 3

**Grupo 6:** 


Ruben Silva - pg57900

Luís Costa - pg55970

# Problema: 


1. Pretende-se um protótipo protocolo $\,{N\choose{N-1}}\,$-OT, usando a abordagem $\,\mathsf{LPN}\,$ “Learning Parity with Noise” [+Capítulo 6d:  Oblivious Linear Evaluation](https://www.dropbox.com/scl/fi/s1kd7552j0vnyrvpdzbzj/Cap-tulo-6d_-Oblivious-Linear-Evaluation.paper?rlkey=48b3owgqxhtww65gsd7aegaj0&dl=0) para mensagens de $n$ bytes (i.e. $8 \times n$ bits) que possa ser convertido para mensagens $\,m\in \mathbb{F}_p^n\,$ (vetores de $n$ componentes no corpo finito  $\,\mathbb{F}_p\,$). Para isso:
    1. Implemente um protótipo do protocolo $\,\mathsf{LPN}$ $\,{N\choose{N-1}}$-OT  para mensagens de $\,n\,$ bytes (i.e. $8 \times n$ bits). Ver [+Capítulo 6d:  Oblivious Linear Evaluation](https://www.dropbox.com/scl/fi/s1kd7552j0vnyrvpdzbzj/Cap-tulo-6d_-Oblivious-Linear-Evaluation.paper?rlkey=48b3owgqxhtww65gsd7aegaj0&dl=0).
    2. Codificando os elementos de um corpo primo $\;\mathbb{F}_p\;$ em “arrays” de “bytes” , converta a solução anterior num protocolo $\,{N\choose{N-1}}$-OT em que as mensagens são  vetores $\,\mathbb{F}_p^\ell\,$.


### imports

In [1]:
from sage.all import GF, vector, random_vector, ZZ
import random
from sage import *


### Funções auxiliares

### Gerador Bernoulli

$\mathcal{B}(\epsilon) \;\equiv\;\vartheta \,w\gets \{0,1\}^n\,\centerdot\,\mathsf{if}\,\;\sum_{i=1}^n\,w_i\,2^{-i}\,\leq\, \varepsilon\;\,\mathsf{then}\,\;1\;\,\mathsf{else}\,\;0$

In [2]:
def bernoulli_generator(epsilon, n=64):
    w = [random.randint(0, 1) for _ in range(n)]
    hat_w = sum(w[i] * (2**-(i+1)) for i in range(n))
    return 1 if hat_w <= epsilon else 0

In [3]:
def error(l, epsilon):
    F2 = GF(2)
    e = [F2(0)] * l
    for i in range(l):
        e[i] = bernoulli_generator(epsilon)
    return e

In [4]:
def gerar_sk(size):
    F2 = GF(2)
    return [F2(random.randint(0, 1)) for _ in range(size)]


## Explicação do Protocolo: 

`Choose`($b$):
* Neste protocolo $b \in [N]$ denota o índice da mensagem que vai ser excluída das transferências legítimas; o criptograma $c_b$ não pode ser decifrado corretamente pelo **receiver** porque este agente não conhece uma chave privada que o permita.

* O **sender** escolhe o par ($\alpha$,$l$) e a função XOF e envia essa informação para o receiver; esta informação determina completamente a sequência $\{(a_i, u_i)\}_{i=1}^l$ que passa a formar o "obvilious criterion"; ambos os asgentes podem construir estes elementos.

1. o **receiver** gera $N$ segredos $s_k \leftarrow \mathcal{B}^\lambda$, se $k \neq b$, e $s_b \leftarrow \bot :$

    1. $\forall k \in [N]$ e $\forall i \in [l]:$
    $$ t_{i,k}\;\gets\;\left\{\begin{array}{lcl}\vartheta\,e_{i,k}\gets \mathcal{B(\epsilon)}\,\centerdot\, a_i\cdot \mathsf{s}_k + e_{i,k} & \text{se} & k\neq b \\ u_i - \sum_{j\neq b}\,t_{i,j} &\text{se}& k=b\end{array}\right.$$
Regista esta informação na sua memória. 
    
2. Construímos, para cada $\,i\in[l]\,$ , um vetor em $\,\mathcal{B}^N$
    $$\mathsf{t}_i\,\equiv\,\{t_{i,k} \;|\;k\in [N]\}\,$$
    e envia-os para o **sender** como chaves públicas.

3. o sender   recolhe todas os vetores de chaves públicas $\,\mathsf{t_i}\,$ e verifca as igualdades: 
$$\sum_{k\in [N]}\,\mathsf{t}_{i,k}\;=\; u_i$$ 
Se, para algum $\,i\in[l]\,$  a igualdade não se verifica então termina em falha.

Se se verificar a igualdade então  regista todos os $\,\mathsf{t}_i\,$ na sua memória para transferência futura.

In [5]:
def receiver_choose(N, l, lambda_, epsilon, b, a_list, u_list):
    F2 = GF(2)
    s = [gerar_sk(lambda_) if k != b else None for k in range(N)]

    t = [[F2(0) for _ in range(N)] for _ in range(l)]

    for i in range(l):
        sum_t = F2(0)
        for k in range(N):
            if k != b:
                e_ik = F2(bernoulli_generator(epsilon))

                a_i = F2(0)
                for j in range(lambda_):
                    a_i += a_list[i][j] * s[k][j]
                
                t[i][k] = a_i + e_ik
                sum_t += t[i][k]

            else:
                t[i][k] = u_list[i] - sum(t[i][j] for j in range(N) if j != b)
        t[i][b] = u_list[i] - sum_t

    return s, t

In [6]:
def verificar_igualdade(t, u_list):
    F2 = GF(2)
    for i in range(len(t)):
        t_sum = F2(sum(t[i][k] for k in range(len(t[i]))))
        if t_sum != F2(u_list[i]):
            print(f"Verification failed for i={i}: {t_sum} != {u_list[i]}")
            return False
    print("Verification successful.")
    return True

`transfer`$(m_o,m_1,... ,m_{N-1})$

1. O **sender**  conhece as mensagens $\,m_k \in\mathbb{F}_2 \,,\,k\in [N] \;$ e, para cada $\,i\in[l]\,$ as chaves públicas $\,\mathsf{t}_i$
Para as cifrar gera aleatoriamente uma sequência de bits $\,\{\,r_i \gets \mathcal{B}\,\}_
{i=0}^l\,$ com um peso de Hamming (número de bits $1$) limitado a uma parâmetro $\,\delta\,$, e calcula:
$$ \,a \gets \sum_i\,r_i\,a_i\,\quad$$ 
$$\quad c_k \gets m_k + \sum_i\,r_i\cdot\mathsf{t}_{i,k}\quad$$
* (O criptograma é o tuplo  $\,\langle\,a\,,\,c_0\,,\,\cdots\,,\,c_{N-1}\,\rangle$  que é enviado ao **receiver**.)

2. O **receiver**  conhece os segredos $\,\mathsf{s}_k\,$  para todo $\,k\in[N]$ . Sabe que $\,\mathsf{s}_b = \bot\,$ e que para todo $\,k\neq b\,$ pode calcular $\,a\cdot \mathsf{s}_k\,$. Sabe também que se verifica, para todo $\,k\neq b\,$, a relação:
$$m_k \;=\; c_k + (a\cdot\mathsf{s}_k) + \mathsf{error}_k$$

In [7]:
def sender_transfer(m, t, a_list, l, N, delta):
    F2 = GF(2)
    # calcular o vetor de erro
    r = error(l, delta)

    # Calcular vetor a
    a = vector(F2, [0] * len(a_list[0]))
    for i in range(l):
        a+= r[i] * a_list[i]

    # Calcular vetor c
    c = []
    for k in range(N):
        c_k = m[k]
        for i in range(l):
            if r[i] == F2(1):
                c_k += t[i][k] * r[i]
        c.append(c_k)
    return a, c

In [8]:
def receiver_decipher(a, c, s, b, l, N, epsilon):
    F2 = GF(2)
    m_recovered = [None] * N
    for k in range(N):
        if k != b and s[k] is not None:
            a_sk = a * vector(F2, s[k])
            m_recovered[k] = c[k] + a_sk 
    return m_recovered

Procedendo como no protocolo ${2\choose{}1}$-OT , pode-se reforçar esta probabilidade, iterando ambas as operações $\,t\,$ vezes; para isso cifra-se usando  vetores $\,r_i\in \mathcal{B}^t\,$. As iterações são independentes e podem ser executadas em paralelo. A sender produz $\,t\,$ criptogramas distintos, um por iteraçao. 
O receiver toma este conjunto de criptogramas e calcula, para cada um, um resultado:

$$m_k \gets c_k + (a\cdot s_k)\quad, \forall k \neq b$$ 


Toma-se como resultado final de $\,m_\kappa\,$ , para cada $k\neq b\,$, o valor em maioria nas diferentes iterações; assim  obtém-se, com elevada probabilidade,  o valor da mensagem inicial.


Finalmente para mensagens $\,\{m_k\}_{k\in[N]}\,$ de comprimento arbitrário, tal como no caso, $\,{2\choose{}1}$ OT, usa-se o protocolo de mensagens binárias para cada posição nas mensagens.

In [9]:
def recuperar_bits(s, b, a_list, t, m, bits_per_message, N, l, t_iterations, delta, epsilon):
    # Armazenar as estimativas dos bits recuperados
    results = [[[] for _ in range(bits_per_message)] for _ in range(N)]

    for bit_pos in range(bits_per_message):

        # Extrai o bit na posição bit_pos de cada mensagem 
        m_bit = [m[k][bit_pos] for k in range(N)]

        for _ in range(t_iterations):

            a, c = sender_transfer(m_bit, t, a_list, l, N, delta)
            m_rec = receiver_decipher(a, c, s, b, l, N, epsilon)

            #Itera sobre cada mensagem k para armazenar a estimativa do bit
            for k in range(N):
                if m_rec[k] is not None:
                    results[k][bit_pos].append(m_rec[k])
    return results  

In [10]:
def votacao(b,resultado, bits_per_message, N):
    F2 = GF(2)

    # Armazenar as estimativas dos bits recuperados
    m_final = [[None] * bits_per_message for _ in range(N)]
    for k in range(N):

        #conforme o protocolo, sb = NULL
        if k != b:
            for bit_pos in range(bits_per_message):
                # Conta o número de estimativas iguais a 1 para o bit
                ones = sum(1 for bit in resultado[k][bit_pos] if bit == 1)

                # Conta o número de estimativas iguais a 0 para o bit
                zeros = len(resultado[k][bit_pos]) - ones

                # Se o número de estimativas iguais a 1 for maior que o número de estimativas iguais a 0,
                # define o bit recuperado como 1, caso contrário, define como 0
                m_final[k][bit_pos] = F2(1) if ones > zeros else F2(0)
    return m_final

### Ex1

In [11]:
def ex1(N, l, lambda_, epsilon, delta, b, t_iterations, n_bytes=1):
    F2 = GF(2)
    # Gerar 
    a_list = [random_vector(F2, lambda_) for _ in range(l)] #Parte do Oblivious Criterion
    u_list = [F2(random.randint(0, 1)) for _ in range(l)] #Parte para verificar as chaves públicas
    
    # n_bytes * 8 bits (PEDIDO PELO EXERCICIO)
    bits_per_message = n_bytes * 8
    m = [[F2(random.randint(0, 1)) for _ in range(bits_per_message)] for _ in range(N)]
    print(f"Mensagem Original: {m}")
    
    # Receiver's choose
    s, t = receiver_choose(N, l, lambda_, epsilon, b, a_list, u_list)
    
    # Sender's verification
    if not verificar_igualdade(t, u_list):
        return False, None
    
    # Tentar recuperar os bits
    resultado = recuperar_bits(s, b, a_list, t, m, bits_per_message, N, l, t_iterations, delta, epsilon)
    
    # Votação para decidir o bit
    m_final = votacao(b, resultado, bits_per_message, N)
    
    print(f"Mensagem Recuperada: {m_final}")
    
    # Verificar
    correct = True
    for k in range(N):
        if k != b:
            if m_final[k] != m[k]:
                print(f"Error: m_{k} incorrect, expected {m[k]}, got {m_final[k]}")
                correct = False
        else:
            if m_final[k] is not None and any(bit is not None for bit in m_final[k]):
                print(f"Error: m_{b} should not be recovered, got {m_final[k]}")
                correct = False
    
    return correct, m_final



In [12]:
N, l, lambda_, epsilon, delta, b, t_iterations = 3, 12, 16, 0.005, 1, 1, 1000
correct, m_final = ex1(N, l, lambda_, epsilon, delta, b, t_iterations)
if correct:
    print("Funcionou!")
else:
    print("Falhou")

Mensagem Original: [[0, 0, 1, 0, 1, 1, 1, 1], [0, 1, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 1, 0, 0, 1]]
Verification successful.
Mensagem Recuperada: [[0, 0, 1, 0, 1, 1, 1, 1], [None, None, None, None, None, None, None, None], [1, 1, 0, 0, 1, 0, 0, 1]]
Funcionou!


### Ex2

In [13]:
# Funções de conversão para F_p^ell
def bytes_to_fp_vector(bytes_msg, p, ell):
    Fp = GF(p)

    # Caso a mensagem seja menor que ell, preenche com 0 (\x00)
    if len(bytes_msg) < ell:
        bytes_msg = bytes_msg.ljust(ell, b'\x00')

    # Caso a mensagem seja maior que ell, corta
    return vector(Fp, [bytes_msg[i] % p for i in range(ell)])

def fp_vector_to_bytes(vector, p):

    # Converte o vetor em uma sequência de bytes
    #este valor pode ser reajustado em numeros de base 2
    return bytes([int(x) % 256 for x in vector])

In [14]:
def ex2(N, l, lambda_, epsilon, delta, b, t_iterations, p, ell):
    F2 = GF(2)
    Fp = GF(p)
    a_list = [random_vector(F2, lambda_) for _ in range(l)]
    u_list = [F2(random.randint(0, 1)) for _ in range(l)]
    
    # Gerar mensagens em F_p^ell
    m_fp = [vector(Fp, [Fp(random.randint(0, p-1)) for _ in range(ell)]) for _ in range(N)]
    print(f"Mensagem Original (F_p): {m_fp}")
    
    # Converter para bytes para protocolo binário
    m_bytes = [fp_vector_to_bytes(m_k, p) for m_k in m_fp]
    bits_per_message = ell * 8
    m = [[F2((m_bytes[k][i // 8] >> (7 - (i % 8))) & 1) for i in range(bits_per_message)] for k in range(N)]
    
    # Receiver's choose
    s, t = receiver_choose(N, l, lambda_, epsilon, b, a_list, u_list)
    
    # Sender's verification
    if not verificar_igualdade(t, u_list):
        return False, None
    
    # Recuperar bits
    resultado = recuperar_bits(s, b, a_list, t, m, bits_per_message, N, l, t_iterations, delta, epsilon)
    
    # Votação
    m_final = votacao(b, resultado, bits_per_message, N)
    
    # Converter bits recuperados para F_p^ell
    m_final_fp = [None] * N
    for k in range(N):
        if k != b and m_final[k] is not None:
            bytes_rec = bytearray(ell)
            for i in range(bits_per_message):
                if m_final[k][i] is not None:
                    byte_idx = i // 8
                    bit_idx = 7 - (i % 8)
                    bytes_rec[byte_idx] |= (int(m_final[k][i]) << bit_idx)
            m_final_fp[k] = bytes_to_fp_vector(bytes_rec, p, ell)
    
    print(f"Mensagem Recuperada (F_p): {m_final_fp}")
    
    # Verificar
    correct = True
    for k in range(N):
        if k != b:
            if m_final_fp[k] != m_fp[k]:
                print(f"Error: m_{k} incorrect, expected {m_fp[k]}, got {m_final_fp[k]}")
                correct = False
        else:
            if m_final_fp[k] is not None:
                print(f"Error: m_{b} should not be recovered, got {m_final_fp[k]}")
                correct = False
    
    return correct, m_final_fp


In [15]:
p, ell = 257, 4
N, l, lambda_, epsilon, delta, b, t_iterations = 3, 12, 16, 0.005, 1, 1, 1000
correct, m_final = ex2(N, l, lambda_, epsilon, delta, b, t_iterations, p, ell)
if correct:
    print("Funcionou!")
else:
    print("Falhou")

Mensagem Original (F_p): [(244, 236, 244, 189), (85, 116, 204, 204), (14, 104, 192, 220)]
Verification successful.
Mensagem Recuperada (F_p): [(244, 236, 244, 189), None, (14, 104, 192, 220)]
Funcionou!
